In [1]:
import os
import cv2
import json
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
from skimage.color import rgb2lab, deltaE_cie76

# Predefined color palette
color_palette = {
    'black': [0, 0, 0],
    'blue': [0, 0, 255],
    'brown': [150, 75, 0],
    'green': [0, 128, 0],
    'grey': [128, 128, 128],
    'orange': [255, 165, 0],
    'pink': [255, 192, 203],
    'purple': [128, 0, 128],
    'red': [255, 0, 0],
    'white': [255, 255, 255],
    'yellow': [255, 255, 0]
}

# Convert RGB palette to LAB for accurate color matching
palette_lab = {name: rgb2lab(np.uint8([[rgb]])) for name, rgb in color_palette.items()}

# Helper function to find the closest color in the palette
def get_closest_color_name(lab_color):
    min_distance = float('inf')
    closest_color = None
    for color_name, color_lab in palette_lab.items():
        distance = deltaE_cie76(lab_color, color_lab)
        if distance < min_distance:
            min_distance = distance
            closest_color = color_name
    return closest_color

# Function to extract dominant colors and compute bounding boxes for each cluster
def extract_dominant_colors_and_bboxes(image, num_clusters=5):
    h, w, _ = image.shape

    # Reshape image into a list of pixels
    reshaped_image = image.reshape((-1, 3))

    # Use KMeans to cluster the pixel colors
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(reshaped_image)

    # Get the labels for each pixel and the cluster centers (dominant colors)
    labels = kmeans.labels_
    dominant_colors = kmeans.cluster_centers_.astype(int)

    # Convert dominant colors to LAB space for accurate matching
    dominant_colors_lab = rgb2lab(np.uint8([[dominant_colors]]).reshape(-1, 1, 3))

    # Find the closest color name for each dominant color
    color_names = [get_closest_color_name(color) for color in dominant_colors_lab]

    # Initialize list to store bounding boxes for each dominant color
    bounding_boxes = []

    # Iterate over each cluster (dominant color)
    for cluster_idx in range(num_clusters):
        # Get the indices of pixels that belong to the current cluster
        cluster_pixels = np.where(labels == cluster_idx)[0]

        # Convert the flat pixel indices to 2D coordinates (y, x)
        pixel_y = cluster_pixels // w
        pixel_x = cluster_pixels % w

        # Calculate the bounding box around the cluster's pixels
        xmin, xmax = pixel_x.min(), pixel_x.max()
        ymin, ymax = pixel_y.min(), pixel_y.max()

        # Add the bounding box and the corresponding color name to the list
        bounding_boxes.append({
            "bounding_box": {
                "xmin": int(xmin),
                "ymin": int(ymin),
                "width": int(xmax - xmin),
                "height": int(ymax - ymin)
            },
            "color_name": color_names[cluster_idx]
        })

    return bounding_boxes

# Function to save the bounding boxes and dominant colors of the image to JSON
def save_dominant_colors_to_json(image, output_path, num_clusters=5):
    # Extract the bounding boxes and dominant colors
    bounding_boxes = extract_dominant_colors_and_bboxes(image, num_clusters=num_clusters)

    # Prepare JSON output
    json_data = {
        "dominant_colors_bounding_boxes": bounding_boxes
    }

    # Save the JSON to the output file
    with open(output_path, 'w') as f:
        json.dump(json_data, f, indent=4)

# Function to process all images in a directory
def process_images_in_directory(input_base_dir, output_base_dir, num_clusters=5):
    # Ensure the output base directory exists
    os.makedirs(output_base_dir, exist_ok=True)

    # Loop through each video folder in the input base directory
    for video_folder in os.listdir(input_base_dir):
        video_folder_path = os.path.join(input_base_dir, video_folder)
        
        if os.path.isdir(video_folder_path) and '_reduced' not in video_folder:
            # Loop through each keyframe subfolder in the video folder
            for keyframe_subfolder in os.listdir(video_folder_path):
                keyframe_subfolder_path = os.path.join(video_folder_path, keyframe_subfolder)
                
                if os.path.isdir(keyframe_subfolder_path):
                    # Loop through all images in the keyframe subfolder
                    for image_filename in tqdm(os.listdir(keyframe_subfolder_path)):
                        if image_filename.endswith(('.jpg', '.png', '.jpeg')):  # Check for valid image extensions
                            image_path = os.path.join(keyframe_subfolder_path, image_filename)
                            image = cv2.imread(image_path)
                            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                            # Generate the JSON output path for the current image
                            output_json_path = os.path.join(output_base_dir, video_folder, keyframe_subfolder)
                            os.makedirs(output_json_path, exist_ok=True)
                            output_json_path = os.path.join(output_json_path, os.path.splitext(image_filename)[0] + '.json')

                            # Process the image and save the results in JSON format
                            save_dominant_colors_to_json(image, output_json_path, num_clusters=num_clusters)
                            print(f"Processed {image_filename}, saved output to {output_json_path}")

# Example usage: Set the input and output directories
input_directory = '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe'
output_directory = '/Users/huyenphung/Desktop/senior-project/keyframe_information/color_metadata'

# Process all images in the directory and produce JSON files with bounding boxes around color clusters
process_images_in_directory(input_directory, output_directory, num_clusters=5)


 10%|█         | 1/10 [00:00<00:03,  2.35it/s]

Processed keyframe_1_0.000000.jpg, saved output to /Users/huyenphung/Desktop/senior-project/keyframe_information/color_metadata/Video2/L02/keyframe_1_0.000000.json


 20%|██        | 2/10 [00:00<00:03,  2.53it/s]

Processed keyframe_7_6.400000.jpg, saved output to /Users/huyenphung/Desktop/senior-project/keyframe_information/color_metadata/Video2/L02/keyframe_7_6.400000.json


 30%|███       | 3/10 [00:01<00:02,  2.52it/s]

...

Processed keyframe_30_30.960000.jpg, saved output to /Users/huyenphung/Desktop/senior-project/keyframe_information/color_metadata/Video1/L01/keyframe_30_30.960000.json


 98%|█████████▊| 56/57 [00:31<00:00,  1.74it/s]

Processed keyframe_39_40.568333.jpg, saved output to /Users/huyenphung/Desktop/senior-project/keyframe_information/color_metadata/Video1/L01/keyframe_39_40.568333.json


100%|██████████| 57/57 [00:32<00:00,  1.77it/s]

Processed keyframe_31_32.028333.jpg, saved output to /Users/huyenphung/Desktop/senior-project/keyframe_information/color_metadata/Video1/L01/keyframe_31_32.028333.json
